# Werewolf Game

This is a demo of how to use AgentScope to play the Werewolf game, where six agents play against werewolves and villagers. The details can be found in `examples/werewolf/`.

To install AgentScope, please follow the steps in [README.md](https://github.com/alibaba/AgentScope/blob/main/README.md#installation).

First, we need to set the model configs for the agents.

In [1]:
model_configs = [
    {
        "type": "openai",
        "name": "gpt-3.5-turbo",
        "api_key": "xxx",
        "organization": "xxx",
        "generate_args": {
            "temperature": 0.5
        }
    },
    {
        "type": "post_api",
        "name": "my_post_api",
        "api_url": "https://xxx",
        "headers": {
        },
        "json_args": {
        }
    }
]

The configs for each agent are as follows. There are six agents, each playing a role in the game. Here, `DictDialogAgent` is a specific class in AgentScope. The details can be found in `src/agentscope/agents/dict_dialog_agent.py`.

In [2]:
agent_configs = [
    {
        "class": "DictDialogAgent",
        "args": {
            "name": "Player1",
            "sys_prompt": "Act as a player in a werewolf game. You are Player1 and\nthere are totally 6 players, named Player1, Player2, Player3, Player4, Player5 and Player6.\n\nPLAYER ROLES:\nIn werewolf game, players are divided into two werewolves, two villagers, one seer and one witch. Note only werewolves know who are their teammates.\nWerewolves: They know their teammates' identities and attempt to eliminate a villager each night while trying to remain undetected.\nVillagers: They do not know who the werewolves are and must work together during the day to deduce who the werewolves might be and vote to eliminate them.\nSeer: A villager with the ability to learn the true identity of one player each night. This role is crucial for the villagers to gain information.\nWitch: A character who has a one-time ability to save a player from being eliminated at night (sometimes this is a potion of life) and a one-time ability to eliminate a player at night (a potion of death).\n\nGAME RULE:\nThe game is consisted of two phases: night phase and day phase. The two phases are repeated until werewolf or villager win the game.\n1. Night Phase: During the night, the werewolves discuss and vote for a player to eliminate. Special roles also perform their actions at this time (e.g., the Seer chooses a player to learn their role, the witch chooses a decide if save the player).\n2. Day Phase: During the day, all surviving players discuss who they suspect might be a werewolf. No one reveals their role unless it serves a strategic purpose. After the discussion, a vote is taken, and the player with the most votes is \"lynched\" or eliminated from the game.\n\nVICTORY CONDITION:\nFor werewolves, they win the game if the number of werewolves is equal to or greater than the number of remaining villagers.\nFor villagers, they win if they identify and eliminate all of the werewolves in the group.\n\nCONSTRAINTS:\n1. Your response should be in the first person.\n2. This is a conversational game. You should response only based on the conversation history and your strategy.\n\nYou are playing werewolf in this game.\n",
            "model": "gpt-3.5-turbo",
            "use_memory": True
        }
    },
    {
        "class": "DictDialogAgent",
        "args": {
            "name": "Player2",
            "sys_prompt": "Act as a player in a werewolf game. You are Player2 and\nthere are totally 6 players, named Player1, Player2, Player3, Player4, Player5 and Player6.\n\nPLAYER ROLES:\nIn werewolf game, players are divided into two werewolves, two villagers, one seer and one witch. Note only werewolves know who are their teammates.\nWerewolves: They know their teammates' identities and attempt to eliminate a villager each night while trying to remain undetected.\nVillagers: They do not know who the werewolves are and must work together during the day to deduce who the werewolves might be and vote to eliminate them.\nSeer: A villager with the ability to learn the true identity of one player each night. This role is crucial for the villagers to gain information.\nWitch: A character who has a one-time ability to save a player from being eliminated at night (sometimes this is a potion of life) and a one-time ability to eliminate a player at night (a potion of death).\n\nGAME RULE:\nThe game is consisted of two phases: night phase and day phase. The two phases are repeated until werewolf or villager win the game.\n1. Night Phase: During the night, the werewolves discuss and vote for a player to eliminate. Special roles also perform their actions at this time (e.g., the Seer chooses a player to learn their role, the witch chooses a decide if save the player).\n2. Day Phase: During the day, all surviving players discuss who they suspect might be a werewolf. No one reveals their role unless it serves a strategic purpose. After the discussion, a vote is taken, and the player with the most votes is \"lynched\" or eliminated from the game.\n\nVICTORY CONDITION:\nFor werewolves, they win the game if the number of werewolves is equal to or greater than the number of remaining villagers.\nFor villagers, they win if they identify and eliminate all of the werewolves in the group.\n\nCONSTRAINTS:\n1. Your response should be in the first person.\n2. This is a conversational game. You should response only based on the conversation history and your strategy.\n\nYou are playing werewolf in this game.\n",
            "model": "gpt-3.5-turbo",
            "use_memory": True
        }
    },
    {
        "class": "DictDialogAgent",
        "args": {
            "name": "Player3",
            "sys_prompt": "Act as a player in a werewolf game. You are Player3 and\nthere are totally 6 players, named Player1, Player2, Player3, Player4, Player5 and Player6.\n\nPLAYER ROLES:\nIn werewolf game, players are divided into two werewolves, two villagers, one seer and one witch. Note only werewolves know who are their teammates.\nWerewolves: They know their teammates' identities and attempt to eliminate a villager each night while trying to remain undetected.\nVillagers: They do not know who the werewolves are and must work together during the day to deduce who the werewolves might be and vote to eliminate them.\nSeer: A villager with the ability to learn the true identity of one player each night. This role is crucial for the villagers to gain information.\nWitch: A character who has a one-time ability to save a player from being eliminated at night (sometimes this is a potion of life) and a one-time ability to eliminate a player at night (a potion of death).\n\nGAME RULE:\nThe game is consisted of two phases: night phase and day phase. The two phases are repeated until werewolf or villager win the game.\n1. Night Phase: During the night, the werewolves discuss and vote for a player to eliminate. Special roles also perform their actions at this time (e.g., the Seer chooses a player to learn their role, the witch chooses a decide if save the player).\n2. Day Phase: During the day, all surviving players discuss who they suspect might be a werewolf. No one reveals their role unless it serves a strategic purpose. After the discussion, a vote is taken, and the player with the most votes is \"lynched\" or eliminated from the game.\n\nVICTORY CONDITION:\nFor werewolves, they win the game if the number of werewolves is equal to or greater than the number of remaining villagers.\nFor villagers, they win if they identify and eliminate all of the werewolves in the group.\n\nCONSTRAINTS:\n1. Your response should be in the first person.\n2. This is a conversational game. You should response only based on the conversation history and your strategy.\n\nYou are playing villager in this game.\n",
            "model": "gpt-3.5-turbo",
            "use_memory": True
        }
    },
    {
        "class": "DictDialogAgent",
        "args": {
            "name": "Player4",
            "sys_prompt": "Act as a player in a werewolf game. You are Player4 and\nthere are totally 6 players, named Player1, Player2, Player3, Player4, Player5 and Player6.\n\nPLAYER ROLES:\nIn werewolf game, players are divided into two werewolves, two villagers, one seer and one witch. Note only werewolves know who are their teammates.\nWerewolves: They know their teammates' identities and attempt to eliminate a villager each night while trying to remain undetected.\nVillagers: They do not know who the werewolves are and must work together during the day to deduce who the werewolves might be and vote to eliminate them.\nSeer: A villager with the ability to learn the true identity of one player each night. This role is crucial for the villagers to gain information.\nWitch: A character who has a one-time ability to save a player from being eliminated at night (sometimes this is a potion of life) and a one-time ability to eliminate a player at night (a potion of death).\n\nGAME RULE:\nThe game is consisted of two phases: night phase and day phase. The two phases are repeated until werewolf or villager win the game.\n1. Night Phase: During the night, the werewolves discuss and vote for a player to eliminate. Special roles also perform their actions at this time (e.g., the Seer chooses a player to learn their role, the witch chooses a decide if save the player).\n2. Day Phase: During the day, all surviving players discuss who they suspect might be a werewolf. No one reveals their role unless it serves a strategic purpose. After the discussion, a vote is taken, and the player with the most votes is \"lynched\" or eliminated from the game.\n\nVICTORY CONDITION:\nFor werewolves, they win the game if the number of werewolves is equal to or greater than the number of remaining villagers.\nFor villagers, they win if they identify and eliminate all of the werewolves in the group.\n\nCONSTRAINTS:\n1. Your response should be in the first person.\n2. This is a conversational game. You should response only based on the conversation history and your strategy.\n\nYou are playing villager in this game.\n",
            "model": "gpt-3.5-turbo",
            "use_memory": True
        }
    },
    {
        "class": "DictDialogAgent",
        "args": {
            "name": "Player5",
            "sys_prompt": "Act as a player in a werewolf game. You are Player5 and\nthere are totally 6 players, named Player1, Player2, Player3, Player4, Player5 and Player6.\n\nPLAYER ROLES:\nIn werewolf game, players are divided into two werewolves, two villagers, one seer and one witch. Note only werewolves know who are their teammates.\nWerewolves: They know their teammates' identities and attempt to eliminate a villager each night while trying to remain undetected.\nVillagers: They do not know who the werewolves are and must work together during the day to deduce who the werewolves might be and vote to eliminate them.\nSeer: A villager with the ability to learn the true identity of one player each night. This role is crucial for the villagers to gain information.\nWitch: A character who has a one-time ability to save a player from being eliminated at night (sometimes this is a potion of life) and a one-time ability to eliminate a player at night (a potion of death).\n\nGAME RULE:\nThe game is consisted of two phases: night phase and day phase. The two phases are repeated until werewolf or villager win the game.\n1. Night Phase: During the night, the werewolves discuss and vote for a player to eliminate. Special roles also perform their actions at this time (e.g., the Seer chooses a player to learn their role, the witch chooses a decide if save the player).\n2. Day Phase: During the day, all surviving players discuss who they suspect might be a werewolf. No one reveals their role unless it serves a strategic purpose. After the discussion, a vote is taken, and the player with the most votes is \"lynched\" or eliminated from the game.\n\nVICTORY CONDITION:\nFor werewolves, they win the game if the number of werewolves is equal to or greater than the number of remaining villagers.\nFor villagers, they win if they identify and eliminate all of the werewolves in the group.\n\nCONSTRAINTS:\n1. Your response should be in the first person.\n2. This is a conversational game. You should response only based on the conversation history and your strategy.\n\nYou are playing seer in this game.\n",
            "model": "gpt-3.5-turbo",
            "use_memory": True
        }
    },
    {
        "class": "DictDialogAgent",
        "args": {
            "name": "Player6",
            "sys_prompt": "Act as a player in a werewolf game. You are Player6 and\nthere are totally 6 players, named Player1, Player2, Player3, Player4, Player5 and Player6.\n\nPLAYER ROLES:\nIn werewolf game, players are divided into two werewolves, two villagers, one seer and one witch. Note only werewolves know who are their teammates.\nWerewolves: They know their teammates' identities and attempt to eliminate a villager each night while trying to remain undetected.\nVillagers: They do not know who the werewolves are and must work together during the day to deduce who the werewolves might be and vote to eliminate them.\nSeer: A villager with the ability to learn the true identity of one player each night. This role is crucial for the villagers to gain information.\nWitch: A character who has a one-time ability to save a player from being eliminated at night (sometimes this is a potion of life) and a one-time ability to eliminate a player at night (a potion of death).\n\nGAME RULE:\nThe game is consisted of two phases: night phase and day phase. The two phases are repeated until werewolf or villager win the game.\n1. Night Phase: During the night, the werewolves discuss and vote for a player to eliminate. Special roles also perform their actions at this time (e.g., the Seer chooses a player to learn their role, the witch chooses a decide if save the player).\n2. Day Phase: During the day, all surviving players discuss who they suspect might be a werewolf. No one reveals their role unless it serves a strategic purpose. After the discussion, a vote is taken, and the player with the most votes is \"lynched\" or eliminated from the game.\n\nVICTORY CONDITION:\nFor werewolves, they win the game if the number of werewolves is equal to or greater than the number of remaining villagers.\nFor villagers, they win if they identify and eliminate all of the werewolves in the group.\n\nCONSTRAINTS:\n1. Your response should be in the first person.\n2. This is a conversational game. You should response only based on the conversation history and your strategy.\n\nYou are playing witch in this game.\n",
            "model": "gpt-3.5-turbo",
            "use_memory": True
        }
    }
]

We also need to set the prompts for different players in differents states.

In [3]:
"""Used to record prompts, will be replaced by configuration"""


class Prompts:
    """Prompts for werewolf game"""

    to_wolves = (
        "{}, you are werewolves. If you are alone, eliminate a player, else "
        "discuss with your teammates and reach an agreement. Response in the "
        "following format which can be loaded by python json.loads()\n"
        "{{\n"
        '    "thought": "thought",\n'
        '    "speak": "thoughts summary to say to others",\n'
        '    "agreement": "whether the discussion reached an agreement or '
        'not(true/false)"\n'
        "}}"
    )

    to_wolves_vote = (
        "Which player do you vote to kill? Response in the following format "
        "which can be loaded by python json.loads()\n"
        "{{\n"
        '   "thought": "thought" ,\n'
        '   "speak": "player_name"\n'
        "}}"
    )

    to_wolves_res = "The player with the most votes is {}."

    to_witch_resurrect = (
        "{witch_name}, you're witch. Tonight {dead_name} is eliminated. Would "
        "you like to resurrect {dead_name}? Response in the following format "
        "which can be loaded by python json.loads()\n"
        "{{\n"
        '    "thought": "thought",\n'
        '    "speak": "thoughts summary to say",\n'
        '    "resurrect": true/false\n'
        "}}"
    )

    to_witch_poison = (
        "Would you like to eliminate one player? Response in the following "
        "json format which can be loaded by python json.loads()\n"
        "{{\n"
        '    "thought": "thought", \n'
        '    "speak": "False or player_name"\n'
        '    "eliminate": ture/false\n'  
        "}}"
    )

    to_seer = (
        "{}, you're seer. Which player in {} would you like to check "
        "tonight? Response in the following json format which can be loaded "
        "by python json.loads()\n"
        "{{\n"
        '    "thought": "thought" ,\n'
        '    "speak": "player_name"\n'
        "}}"
    )

    to_seer_result = "Okay, the role of {} is {}."

    to_all_danger = (
        "The day is coming, all the players open your eyes. Last night, "
        "the following player(s) has been eliminated: {}."
    )

    to_all_peace = (
        "The day is coming, all the players open your eyes. Last night is "
        "peaceful, no player is eliminated."
    )

    to_all_discuss = (
        "Now the alive players are {}. Given the game rules and your role, "
        "based on the "
        "situation and the information you gain, to vote a player eliminated "
        "among alive players and to win the game, what do you want to say "
        "to others? You can decide whether to reveal your role. Response in "
        "the following JSON format which can be loaded by python json.loads("
        ")\n"
        "{{\n"
        '    "thought": "thought" ,\n'
        '    "speak": "thought summary to say to others"\n'
        "}}"
    )

    to_all_vote = (
        "Now the alive players are {}. Given the game rules and your role, "
        "based on the situation and the information you gain, to vote a "
        "player among alive players and to win the "
        "game, it's time to vote one player among the alive players, please "
        "cast your vote on who you believe is a werewolf. Response in the "
        "following format which can be loaded by python json.loads()\n"
        "{{\n"
        '    "thought": "thought",\n'
        '    "speak": "player_name"\n'
        "}}"
    )

    to_all_res = "{} has been voted out."

    to_all_wolf_win = (
        "The werewolves have prevailed and taken over the village. Better "
        "luck next time!"
    )

    to_all_village_win = (
        "The game is over. The werewolves have been defeated, and the village "
        "is safe once again!"
    )

    to_all_continue = "The game goes on."

During the game, we also need some utility funcitons. We list them below.

In [4]:
"""utils."""
import re
from typing import Union, Any, Sequence

import numpy as np
from loguru import logger

from agentscope.agents import AgentBase
from agentscope.message import Msg


def check_winning(alive_agents: list, wolf_agents: list, host: str) -> bool:
    """check which group wins"""
    if len(wolf_agents) * 2 >= len(alive_agents):
        msg = Msg(host, Prompts.to_all_wolf_win)
        logger.chat(f"{host}: {msg.content}")
        return True
    if alive_agents and not wolf_agents:
        msg = Msg(host, Prompts.to_all_village_win)
        logger.chat(f"{host}: {msg.content}")
        return True
    return False


def update_alive_players(
    survivors: Sequence[AgentBase],
    wolves: Sequence[AgentBase],
    dead_names: Union[str, list[str]],
) -> tuple[list, list]:
    """update the list of alive agents"""
    if not isinstance(dead_names, list):
        dead_names = [dead_names]
    return [_ for _ in survivors if _.name not in dead_names], [
        _ for _ in wolves if _.name not in dead_names
    ]


def majority_vote(votes: list) -> Any:
    """majority_vote function"""
    unit, counts = np.unique(votes, return_counts=True)
    return unit[np.argmax(counts)]


def extract_name_and_id(name: str) -> tuple[str, int]:
    """extract player name and id from a string"""
    name = re.search(r"\bPlayer\d+\b", name).group(0)
    idx = int(re.search(r"Player(\d+)", name).group(1)) - 1
    return name, idx


def n2s(agents: Sequence[Union[AgentBase, str]]) -> str:
    """combine agent names into a string, and use "and" to connect the last two names."""

    def _get_name(agent_: Union[AgentBase, str]) -> str:
        return agent_.name if isinstance(agent_, AgentBase) else agent_

    if len(agents) == 1:
        return _get_name(agents[0])

    return (
        ", ".join([_get_name(_) for _ in agents[:-1]])
        + " and "
        + _get_name(agents[-1])
    )

Now we can turn our attention to the main function of the game. Run the following codes and enjoy the game.

In [ ]:
"""A werewolf game implemented by agentscope."""
from functools import partial
from agentscope.message import Msg
from agentscope.msghub import msghub
from agentscope.pipelines.functional import sequentialpipeline
import agentscope

# default settings
HostMsg = partial(Msg, name="Moderator", echo=True)
healing, poison = True, True
MAX_WEREWOLF_DISCUSSION_ROUND = 3
MAX_GAME_ROUND = 6
# read model and agent configs, and initialize agents automatically
survivors = agentscope.init(
    model_configs=model_configs,
    agent_configs=agent_configs,
)
roles = ["werewolf", "werewolf", "villager", "villager", "seer", "witch"]
wolves, witch, seer = survivors[:2], survivors[-1], survivors[-2]

# start the game
for i in range(1, MAX_GAME_ROUND + 1):
    # night phase, werewolves discuss
    hint = HostMsg(content=Prompts.to_wolves.format(n2s(wolves)))
    with msghub(wolves, announcement=hint) as hub:
        for _ in range(MAX_WEREWOLF_DISCUSSION_ROUND):
            x = sequentialpipeline(wolves)
            if x.get("agreement", False):
                break

        # werewolves vote
        hint = HostMsg(content=Prompts.to_wolves_vote)
        votes = [extract_name_and_id(wolf(hint).content)[0] for wolf in wolves]
        # broadcast the result to werewolves
        dead_player = [majority_vote(votes)]
        hub.broadcast(
            HostMsg(content=Prompts.to_wolves_res.format(dead_player[0])),
        )

    # witch
    healing_used_tonight = False
    if witch in survivors:
        if healing:
            hint = HostMsg(
                content=Prompts.to_witch_resurrect.format_map(
                    {"witch_name": witch.name, "dead_name": dead_player[0]},
                ),
            )
            if witch(hint).get("resurrect", False):
                healing_used_tonight = True
                dead_player.pop()
                healing = False

        if poison and not healing_used_tonight:
            x = witch(HostMsg(content=Prompts.to_witch_poison))
            if x.get("eliminate", True):
                dead_player.append(extract_name_and_id(x.content)[0])
                poison = False

    # seer
    if seer in survivors:
        hint = HostMsg(
            content=Prompts.to_seer.format(seer.name, n2s(survivors)),
        )
        x = seer(hint)

        player, idx = extract_name_and_id(x.content)
        role = "werewolf" if roles[idx] == "werewolf" else "villager"
        hint = HostMsg(content=Prompts.to_seer_result.format(player, role))
        seer.observe(hint)

    survivors, wolves = update_alive_players(survivors, wolves, dead_player)
    if check_winning(survivors, wolves, "Moderator"):
        break

    # daytime discussion
    content = (
        Prompts.to_all_danger.format(n2s(dead_player))
        if dead_player
        else Prompts.to_all_peace
    )
    hints = [
        HostMsg(content=content),
        HostMsg(content=Prompts.to_all_discuss.format(n2s(survivors))),
    ]
    with msghub(survivors, announcement=hints) as hub:
        # discuss
        x = sequentialpipeline(survivors)

        # vote
        hint = HostMsg(content=Prompts.to_all_vote.format(n2s(survivors)))
        votes = [extract_name_and_id(_(hint).content)[0] for _ in survivors]
        vote_res = majority_vote(votes)
        # broadcast the result to all players
        result = HostMsg(content=Prompts.to_all_res.format(vote_res))
        hub.broadcast(result)

        survivors, wolves = update_alive_players(survivors, wolves, vote_res)

        if check_winning(survivors, wolves, "Moderator"):
            break

        hub.broadcast(HostMsg(content=Prompts.to_all_continue))
